In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.3 MB/s eta 0:00:00


In [2]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.0 MB/s eta 0:00:00


In [3]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.4 MB/s eta 0:00:00


In [4]:
import pandas as pd
from sklearn.utils import shuffle
import torch
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, Trainer, TrainingArguments,BertTokenizerFast,AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss
from scipy.special import softmax
from nlp import load_dataset
from nlp import Dataset
import numpy as np
import openai

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [9]:
training_args1 = TrainingArguments(
    output_dir='/content/results', #存储结果文件的目录
    overwrite_output_dir=True,
    num_train_epochs=3, #4
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32, 
    learning_rate=5e-5,
    load_best_model_at_end=True, 
    metric_for_best_model = "accuracy", # 最后载入最优模型的评判标准，这里选用acc最高的那个模型参数
    weight_decay=0.01,
    warmup_steps=1,
    evaluation_strategy="steps", #这里设置每100个batch做一次评估，也可以为“epoch”，也就是每个epoch进行一次
    logging_strategy = "steps",
    save_strategy ='steps',
    logging_steps = 1,
    seed = 2023,
    logging_dir='/content/logs' #存储logs的目录
)
training_args2 = TrainingArguments(
    output_dir='/content/results', #存储结果文件的目录
    overwrite_output_dir=True,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32, 
    learning_rate=5e-5,
    load_best_model_at_end=True, 
    metric_for_best_model = "accuracy", # 最后载入最优模型的评判标准，这里选用acc最高的那个模型参数
    weight_decay=0.01,
    warmup_steps=1,
    evaluation_strategy="steps", #这里设置每100个batch做一次评估，也可以为“epoch”，也就是每个epoch进行一次
    logging_strategy = "steps",
    save_strategy ='steps',
    logging_steps = 1,
    seed = 2023,
    logging_dir='/content/logs' #存储logs的目录
)

training_args = [training_args1,training_args2]

In [10]:
def tokenize_data(X_train,y_train):
  train_dataset = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=max_len)
  max_token = []
  for i in range(len(X_train.tolist())):
    decoded_encodings=tokenizer.tokenize(X_train.tolist()[i])
    max_token.append(len(decoded_encodings))
  print(max(max_token))
  train_dataset['label'] = y_train.values
  return train_dataset

In [11]:
def reply(index_hard,index_simple,train_data):
    change = 0
    for i in index_hard:
      pred_train_label = 0
      count = 0
      while pred_train_label == 0 and count <= 2:
        #print('gen harder',count,'round for data',i)
        answer = support_hard.loc[i,'harder'+str(count)]
        pred_train_label = train_bert(answer,None,None,'test',None) #get the predict label for specific round
        count += 1
      #print(count)
      if count < 3:
        train_data.loc[i,'content'] = answer
        #print(train_data.loc[i,'label'])
        change += 1
      if change >= np.round(len(index_hard)/2):
        break
    print('changing',change,'to hard.')
    # change = 0
    # for i in index_simple:
    #   pred_train_label = 1
    #   count = 0
    #   while pred_train_label == 1 and count <= 2:
    #     #print('gen simpler',count,'round for data',i)
    #     answer = support_simple.loc[i,'simplier'+str(count)]
    #     pred_train_label = train_bert(answer,None,None,'test',None) #get the predict label for specific round
    #     count += 1
    #   if count < 3:
    #     train_data.loc[i,'content'] = answer
    #     change += 1
    #   if change >= np.round(len(index_simple)/2):
    #     break
    # print('changing',change,'to simple.')

In [13]:
train_data = pd.read_csv('/content/train_data.csv',encoding='gb18030') #Read data
init_re = pd.read_csv('/content/init_re.csv',encoding='gb18030')
support_hard = pd.read_csv('/content/harder.csv',encoding='gb18030')
support_simple = pd.read_csv('/content/simplier.csv',encoding='gb18030')
# init_re['label']=0
# init_re['index_article'] = train_data['index_article']
# init_re['index_paragraph'] = train_data['index_paragraph']
train_data = pd.concat([init_re,train_data]).reset_index()

In [17]:
val_data = pd.read_csv('/content/val_data.csv',encoding='gb18030')#need to gen before
test_data = pd.read_csv('/content/test_data.csv',encoding='gb18030')#need to gen before
test_data = test_data.dropna()
seed = 2023
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

X_val = val_data['content'] 

y_val = val_data['label']
#X_train, X_val, y_train, y_val = train_test_split(train_data['content'],train_data['label'], test_size=0.1, random_state=2023)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2,output_hidden_states = False)
max_len = 216
val_dataset = tokenizer(X_val.tolist(), padding=True, truncation=True, max_length=max_len)
val_dataset['label'] = y_val.values
test_dataset = tokenizer(test_data['content'].tolist(),padding=True, truncation=True, max_length=max_len)
val_dataset = Dataset.from_dict(val_dataset)
test_dataset = Dataset.from_dict(test_dataset)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
def train_bert(train_dataset,val_dataset=None,test_dataset=None,status=None,training_args=None):
  global max_len
  trainer = Trainer(model=model,args=training_args,compute_metrics=compute_metrics,train_dataset=train_dataset,eval_dataset=val_dataset,tokenizer=tokenizer)
  if status == 'train':
    trainer.train()
    pred_train = trainer.predict(test_dataset=train_dataset).predictions
    pred = trainer.predict(test_dataset=test_dataset).predictions
    prob_train = softmax(pred_train, axis = 1)
    print('pred:',pred)
    prob = softmax(pred, axis = 1)
    print('prob:',prob)
    pred_label_train = pred_train.argmax(-1)
    pred_label = prob.argmax(-1)
    return pred_label_train,pred_label
  elif status=='test':
    df = pd.DataFrame(columns=['content'], index=[0])
    df.loc[0,'content'] = train_dataset
    train_dataset = tokenizer(df['content'].tolist(),padding=True,truncation=True,max_length=max_len) #input train dataset is an paragraph
    train_dataset = Dataset.from_dict(train_dataset)
    pred_train = trainer.predict(test_dataset=train_dataset).predictions
    prob_train = softmax(pred_train, axis = 1)
    pred_label_train = pred_train.argmax(-1)
    return pred_label_train
  else:
    df = pd.DataFrame(columns=['content'], index=[0])
    df.loc[0,'content'] = train_dataset
    # print('yes',train_dataset)
    train_dataset = tokenizer(df.loc[0,'content'].tolist(),padding=True,truncation=True,max_length=max_len) #input train dataset is an paragraph
    # print(df.loc[0,'content'].tolist())
    # print(train_dataset)
    train_dataset = Dataset.from_dict(train_dataset)
    pred_train = trainer.predict(test_dataset=train_dataset).predictions
    prob_train = softmax(pred_train, axis = 1)
    pred_label_train = pred_train.argmax(-1)
    return pred_label_train


In [19]:
def judge(y_train,y_train_pred):
  index_hard = []
  index_simple = []
  for i in range(len(y_train)):
    if y_train[i] == y_train_pred[i] and y_train[i] == 0:
      index_hard.append(i)
    elif y_train[i] != y_train_pred[i] and y_train[i] == 0:
      index_simple.append(i)
  return index_hard,index_simple


In [20]:
train_data = train_data.dropna()
X_train = train_data['content']
y_train = train_data['label']
train_dataset = tokenize_data(X_train,y_train)
train_dataset = Dataset.from_dict(train_dataset)
train_label,test_label = train_bert(train_dataset,val_dataset,test_dataset,'train',training_args[0])
acc = accuracy_score(test_data['label'], test_label)
precision, recall, f1, _ = precision_recall_fscore_support(test_data['label'], test_label, average='binary')
print('in round 1 testing acc is:',acc)
print(precision,recall,f1)
index_hard, index_simple = judge(train_data['label'],train_label)


406


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.673900,0.712774,0.505051,0.671141,0.505051,1.000000
2,0.737300,0.676706,0.555556,0.241379,0.875000,0.140000
3,0.692300,0.687893,0.494949,0.000000,0.000000,0.000000
4,0.695600,0.687602,0.494949,0.000000,0.000000,0.000000
5,0.696300,0.696811,0.494949,0.000000,0.000000,0.000000
6,0.784000,0.689905,0.474747,0.000000,0.000000,0.000000
7,0.628300,0.676985,0.525253,0.145455,0.800000,0.080000
8,0.645500,0.674662,0.575758,0.343750,0.785714,0.220000
9,0.689600,0.701236,0.424242,0.467290,0.438596,0.500000
10,0.663700,0.719725,0.404040,0.486957,0.430769,0.560000


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


in round 1 testing acc is: 0.7070707070707071
0.7142857142857143 0.7 0.7070707070707072


In [21]:
reply(index_hard,index_simple,train_data)
train_data = train_data.dropna()
X_train = train_data['content']
y_train = train_data['label']
train_dataset = tokenize_data(X_train,y_train)
train_dataset = Dataset.from_dict(train_dataset)
train_label,test_label = train_bert(train_dataset,val_dataset,test_dataset,'train',training_args[1])
acc = accuracy_score(test_data['label'], test_label)
precision, recall, f1, _ = precision_recall_fscore_support(test_data['label'], test_label, average='binary')
print('in round 2 testing acc is:',acc)
print(precision,recall,f1)
test_data['pred']=test_label
test_data.to_csv('/content/predict2_law_data.csv')

changing 19 to hard.
406


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.180000,0.491672,0.767677,0.772277,0.764706,0.780000
2,0.131200,0.533133,0.767677,0.776699,0.754717,0.800000
3,0.180000,0.523137,0.707071,0.738739,0.672131,0.820000
4,0.323900,0.516598,0.737374,0.740000,0.740000,0.740000
5,0.242700,0.522827,0.767677,0.762887,0.787234,0.740000
6,0.235700,0.549255,0.777778,0.775510,0.791667,0.760000


KeyboardInterrupt: ignored